In [226]:
import vrep 
import sys
import time 
import numpy as np
from tank import *

import skfuzzy
from skfuzzy import control as ctrl

In [227]:
def perpendicular_zero_stage(distance_NW_input, distance_WN_input, distance_NE_input,  tank):
    distance_WN = ctrl.Antecedent(np.arange(0, 5, 0.01), 'distance_WN')
    distance_NW = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance_NW')
    distance_NE = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance_NE')

    new_speed = ctrl.Consequent(np.arange(-1, 7, 0.01), 'new_speed')

    distance_WN['poor'] = skfuzzy.trapmf(distance_WN.universe, [0, 0, 1.5, 3])
    distance_WN['good'] = skfuzzy.trapmf(distance_WN.universe, [1.5, 2.51, 5, 5])

    distance_NW['poor'] = skfuzzy.trapmf(distance_NW.universe, [0, 0, 1.5, 2])
    distance_NW['good'] = skfuzzy.trapmf(distance_NW.universe, [1, 2, 4, 5])

    distance_NE['poor'] = skfuzzy.trapmf(distance_NW.universe, [0, 0, 1.5, 2])
    distance_NE['good'] = skfuzzy.trapmf(distance_NW.universe, [1, 2, 4, 5])

    new_speed['zero'] = skfuzzy.trimf(new_speed.universe, [-1, 0, 1])
    new_speed['poor'] = skfuzzy.trimf(new_speed.universe, [0, 2, 4])
    new_speed['good'] = skfuzzy.trimf(new_speed.universe, [3, 5, 7])

    # dismal poor mediocre average decent good excellent
    rules = (
    ctrl.Rule(distance_WN['poor'] & distance_NW['poor'] & distance_NE['poor'], new_speed['good']),
    ctrl.Rule(distance_WN['good'] & distance_WN['poor'] & distance_NE['good'], new_speed['zero']),
    ctrl.Rule(distance_WN['poor'] & distance_NW['good'], new_speed['good']),
    ctrl.Rule(distance_WN['good'] & distance_NW['good'], new_speed['good'])
    )

    driving_ctrl = ctrl.ControlSystem(rules)
    driving = ctrl.ControlSystemSimulation(driving_ctrl)

    driving.input['distance_NW'] = distance_NW_input
    driving.input['distance_WN'] = distance_WN_input
    driving.input['distance_NE'] = distance_NE_input
    driving.compute()

    return driving.output['new_speed']

def perpendicular_first_stage(distance_WN_input, distance_EN_input, distance_WS_input, distance_ES_input,  tank):
    distance_WN = ctrl.Antecedent(np.arange(0, 5, 0.01), 'distance_WN')
    distance_EN = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance_EN')
    distance_WS = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance_WS')
    distance_ES = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance_ES')

    new_speed = ctrl.Consequent(np.arange(-1, 3, 0.01), 'new_speed')

    distance_WN['poor'] = skfuzzy.trapmf(distance_WN.universe, [0, 0, 1, 2])
    distance_WN['good'] = skfuzzy.trapmf(distance_WN.universe, [1.5, 2.51, 5, 5])

    distance_EN['poor'] = skfuzzy.trapmf(distance_EN.universe, [0, 0, 1.5, 2])
    distance_EN['good'] = skfuzzy.trapmf(distance_EN.universe, [1, 2, 3, 6])

    distance_WS['poor'] = skfuzzy.trapmf(distance_WS.universe, [0, 0, 1.5, 2])
    distance_WS['good'] = skfuzzy.trapmf(distance_WS.universe, [1, 2, 3, 6])

    distance_ES['poor'] = skfuzzy.trapmf(distance_ES.universe, [0, 0, 1.5, 2])
    distance_ES['good'] = skfuzzy.trapmf(distance_ES.universe, [1, 2, 3, 6])

    new_speed['zero'] = skfuzzy.trimf(new_speed.universe, [-1, 0, 1])
    new_speed['poor'] = skfuzzy.trimf(new_speed.universe, [0, 1, 2])
    new_speed['good'] = skfuzzy.trimf(new_speed.universe, [1, 2, 3])


    # dismal poor mediocre average decent good excellent
    rules = (
    ctrl.Rule(distance_WN['good'] & distance_EN['poor'], new_speed['good']),
    ctrl.Rule(distance_WN['good'] & distance_EN['good'], new_speed['poor']),
    ctrl.Rule(distance_WN['poor'] & distance_EN['poor'], new_speed['poor']),
    ctrl.Rule(distance_WN['poor'] & distance_EN['good'], new_speed['poor']),

    ctrl.Rule(distance_WS['good'] & distance_ES['poor'], new_speed['good']),
    ctrl.Rule(distance_WS['good'] & distance_ES['good'], new_speed['poor']),
    ctrl.Rule(distance_WS['poor'] & distance_ES['poor'], new_speed['zero']),
    ctrl.Rule(distance_WS['poor'] & distance_ES['good'], new_speed['poor']),
    )

    driving_ctrl = ctrl.ControlSystem(rules)
    driving = ctrl.ControlSystemSimulation(driving_ctrl)

    driving.input['distance_WN'] = distance_WN_input
    driving.input['distance_EN'] = distance_EN_input
    driving.input['distance_ES'] = distance_ES_input
    driving.input['distance_WS'] = distance_WS_input

    driving.compute()

    return driving.output['new_speed']

In [228]:
vrep.simxFinish(-1) # closes all opened connections, in case any prevoius wasnt finished
clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # start a connection

if clientID!=-1:
    print ("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

#create instance of Tank
tank=Tank(clientID)

Connected to remote API server


In [229]:
proximity_sensors=["EN","ES","NE","NW","SE","SW","WN","WS"]
proximity_sensors_handles=[0]*8

# get handle to proximity sensors
for i in range(len(proximity_sensors)):
    err_code,proximity_sensors_handles[i] = vrep.simxGetObjectHandle(clientID,"Proximity_sensor_"+proximity_sensors[i], vrep.simx_opmode_blocking)
    
#read and print values from proximity sensors
#first reading should be done with simx_opmode_streaming, further with simx_opmode_buffer parameter
for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_streaming)

In [230]:
from time import sleep

#continue reading and printing values from proximity sensors
distances = dict()
detection_states = dict()

for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer)
        distances[sensor_name] = np.linalg.norm(detectedPoint)
        detection_states[sensor_name] = detectionState

current_stage = 0
t = time.time()
while (time.time()-t)<100: # read values for 5 seconds

    for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer)
        distances[sensor_name] = np.linalg.norm(detectedPoint)
        detection_states[sensor_name] = detectionState

    if current_stage == 0:
        new_speed = perpendicular_zero_stage(distances['NW'], distances['WN'], distances['NE'], tank)
        if new_speed < 3.45:
            tank.stop()
            # sleep(10)
            current_stage += 1
        else:
            tank.forward(new_speed)

    if current_stage == 1:
        distances['WN'] = distances['WN'] if detection_states['WN'] else 5
        distances['EN'] = distances['EN'] if detection_states['EN'] else 5
        distances['WS'] = distances['WS'] if detection_states['WS'] else 5
        distances['ES'] = distances['ES'] if detection_states['ES'] else 5

        new_speed = perpendicular_first_stage(distances['WN'], distances['EN'], distances['WS'], distances['ES'], tank)
        if new_speed < 0.51:
            tank.forward(0)
            # sleep(10)
            current_stage += 1
        else:
            tank.turn_left(new_speed*1.075)
            sleep(0.1)
            tank.forward(new_speed*2)
            sleep(0.2)

    if current_stage == 2:
        tank.forward(3)
        if not detection_states['NW'] and not detection_states['NE']:
            tank.forward(0)
            break
    #
    # if current_stage == 2:
    #     new_speed = parallel_second_stage(distances['EN'], distances['ES'], distances['SE'], distances['NE'], tank)
    #     if new_speed < 1:
    #         tank.forward(0)
    #         current_stage += 1
    #     else:
    #         tank.leftvelocity = -new_speed
    #         tank.rightvelocity = -new_speed / 6
    #         tank.setVelocity()

    print(f"\r{new_speed, tank.readVelocity(), distances['WN'], distances['NW'], distances['WS'], distances['ES'], current_stage}", end='')

(0.5000000000000003, 3.0, 0.4345043152303278, 0.5983356706936205, 0.4308830428501173, 0.6064529501450796, 2)))32393988, 1))